In [1]:
# pip install llama-cpp-python, langchain, llamaindex, chromadb, sentence_transformers, pypdf
#CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [1]:
#mistral-7b-instruct-v0.2.Q4_K_M.gguf

from llama_index.llms import LlamaCPP

In [2]:
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."

#"You can only answer questions about the provided context. If you know the answer but it is not based in the provided context, don't provide the answer, just state the answer is not in the context provided."

#"You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."


In [3]:
from llama_index.llms.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

llm = LlamaCPP(
                    model_path=str("./mistral-7b-instruct-v0.2.Q4_K_M.gguf"),
                    temperature=0,
                    max_new_tokens=512,
                    system_prompt=system_prompt,
                    context_window=3900, #context_window=8192, # Mistral7B has an 8K context-window
                    generate_kwargs={},
                    # All to GPU
                    model_kwargs={"n_gpu_layers": -1},
                    verbose=True,
                    #messages_to_prompt=messages_to_prompt,
                    #completion_to_prompt=completion_to_prompt
                )

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA TITAN RTX, compute capability 7.5, VMM: yes
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  

In [4]:

from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import ServiceContext, set_global_service_context

# Will download Embedding Model from Huggingfaces
embed_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5"
)

# chunk documents into tokens with embedding model, and use llm to generate responses
service_context = ServiceContext.from_defaults(
    chunk_size=1024, # number of tokens per chunk -> bigger chunk = bigger source snippets
    llm=llm,
    embed_model=embed_model
)

In [5]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
import chromadb
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
import os

documents = SimpleDirectoryReader("main_data").load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)


'''
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("pruefungsamt")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

if not os.path.exists('./chroma_db/'):
    documents = SimpleDirectoryReader("./main_data/").load_data()
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        service_context=service_context
    )
    
index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)
'''

'\ndb = chromadb.PersistentClient(path="./chroma_db")\nchroma_collection = db.get_or_create_collection("pruefungsamt")\nvector_store = ChromaVectorStore(chroma_collection=chroma_collection)\nstorage_context = StorageContext.from_defaults(vector_store=vector_store)\n\nif not os.path.exists(\'./chroma_db/\'):\n    documents = SimpleDirectoryReader("./main_data/").load_data()\n    index = VectorStoreIndex.from_documents(\n        documents,\n        storage_context=storage_context,\n        service_context=service_context\n    )\n    \nindex = VectorStoreIndex.from_vector_store(\n    vector_store,\n    service_context=service_context,\n)\n'

In [6]:
query_engine = index.as_query_engine(streaming=True)
response = query_engine.query("Was sind Zulassungsvorraussetzungen für den Master Informatik?")
response.print_response_stream()


The admission requirements for the Master's program in Informatics include:
1. A completed undergraduate degree with a minimum grade of 2.4 in one of the following fields: Informatics, Business Information Technology, Automotive Information Technology, or a related field. For a related field, at least 60 ECTS points must be earned in Informatics-related subjects.
2. A choice of one of the three study directions: Autonomous Systems (MSI-AS), IT Management (MSI-ITM), or Software Engineering (MSI-SE).
3. Successful participation in a selection interview, which is evaluated based on motivation for the chosen field of study, communication skills, technical understanding, and personal impression. The evaluation criteria are rated on a scale from 1 to 5, with intermediate values allowed by rounding up or down. A candidate must receive at least a 4.0 in all categories to be successful in the interview. The average of these ratings forms the first part of the selection decision.
4. The average


llama_print_timings:        load time =    3594.98 ms
llama_print_timings:      sample time =     113.93 ms /   292 runs   (    0.39 ms per token,  2562.93 tokens per second)
llama_print_timings: prompt eval time =   26253.01 ms /  2067 tokens (   12.70 ms per token,    78.73 tokens per second)
llama_print_timings:        eval time =   29378.76 ms /   291 runs   (  100.96 ms per token,     9.91 tokens per second)
llama_print_timings:       total time =   56685.73 ms


In [7]:
for index, node in enumerate(response.source_nodes, start=1):
    print(f"{index}/{len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")
    print(node.node.metadata)

1/2: |119_ZuSMa_Senat_18012022.pdf| Hochschule Konstanz  
Technik, Wirtschaft und Gestaltung  
 
 
Seite 21 von 43 § 17 Informatik (MSI)  
(1) Studiengangspezifische Zugangsvoraussetzungen gemäß § 5  Abs. 1  
Zugangsvoraussetzung für den Masterstudiengang Informatik ist ein mit mindestens der Note 
2,4 abgeschlossenes Hochschulstudium der Informatik, Wirtsc haftsinformatik, 
Automobilinformationstechnik, Gesundheitsinformatik oder einer verwandten Fachrichtung. Bei einer verwandten Fachrichtung müssen mindestens 60 ECTS -Punkte in Fächern der 
Informatik nachgewiesen werden. Mit dem Antrag auf Zulassung zum Stud ium muss eine der 
drei Studienrichtungen verbindlich gewählt werden: Autonome Systeme (MSI -AS), IT -
Management (MSI -ITM) oder Software- Engineering (MSI -SE). Auf die einzelnen 
Studienrichtungen entfallen in der Regel jeweils ein Drittel der zur Verfügung stehenden Studienplätze.  
(2) Auswahlkriterien nach § 9 Abs. 2  
1. Ergebnis eines Auswahlgesprächs  
Die Bewertung de